## Compiling Pre-Trained Models from keras
In this notebook, we illustrate how we can use `nnexpress` to compile pre-trained models from the keras framework. The notebook illustrates how to load a model, convert it to tflite, save the tflite file, and compile it for n2x sw and hw options. The code is tested on multiple models which can be chosen by uncommenting the appropriate line.

In [1]:
#imports for tensorflow and keras models
import tensorflow as tf
from tensorflow.keras.applications import MobileNet, MobileNetV2, MobileNetV3Small, MobileNetV3Large
from tensorflow.keras.applications import ResNet50, ResNet50V2, ResNet101, ResNet152, ResNet152V2
#imports for nnexpress compiler and utilities
import nnexpress.compiler as n2xc
from nnexpress_utils import run_n2x, compare_n2x_sw_hw, estimate_fps

2022-10-06 18:41:46.703964: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-06 18:41:46.792648: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
model=MobileNet(input_shape=(224,224,3),weights='imagenet',include_top=False, pooling='avg')
# model=MobileNetV2(input_shape=(224,224,3),weights='imagenet',)
# model=MobileNetV3Small(input_shape=(224,224,3),weights='imagenet',minimalistic=True, 
#                        include_top=False,pooling='avg', include_preprocessing=False)
# model=MobileNetV3Large(input_shape=(224,224,3),weights='imagenet',minimalistic=True, 
#                        include_top=False,pooling='avg', include_preprocessing=False)
# model=ResNet50(input_shape=(224,224,3),weights='imagenet',)
# model=ResNet50V2(input_shape=(224,224,3),weights='imagenet',include_top=False,pooling='avg')
# model=ResNet101(input_shape=(224,224,3),weights='imagenet',)
# model=ResNet152(input_shape=(224,224,3),weights='imagenet',)
# model=ResNet152V2(input_shape=(224,224,3),weights='imagenet',include_top=False,pooling='avg')

2022-10-06 18:41:57.419336: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model) 
tflite_model = converter.convert()
# Save the model.
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: /tmp/tmp8opy6obz/assets


INFO:tensorflow:Assets written to: /tmp/tmp8opy6obz/assets
2022-10-06 18:42:06.647013: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-10-06 18:42:06.647045: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-10-06 18:42:06.647987: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp8opy6obz
2022-10-06 18:42:06.664364: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-10-06 18:42:06.664416: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp8opy6obz
2022-10-06 18:42:06.696578: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-10-06 18:42:06.708047: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-10-06 18:42:06.934132: I tensorflow/cc/saved_model/loader.cc:213] Running initializatio

In [4]:
tflite_path='model.tflite'
n2x_sw_path='model_sw.n2x'
n2x_hw_path='model_hw.n2x'

In [5]:
sw_model=n2xc.Compiler(tflite_path, device='SW')
hw_model=n2xc.Compiler(tflite_path, device='HW')
sw_model.save(n2x_sw_path)
hw_model.save(n2x_hw_path)

**Note**: This below cells can run only on a device equipped with ORCA

In [6]:
match=compare_n2x_sw_hw(n2x_sw_path,n2x_hw_path)
if match:
    print('N2X SW and HW match')
else:
    print('N2X SW and HW do not match')    

N2X SW and HW match


In [7]:
fps=estimate_fps(n2x_hw_path)
print('FPS=',fps)

FPS= 75.4300133814654
